<a href="https://colab.research.google.com/github/HadasRavikovitch/Final-Project---GPU/blob/main/cross_time_tests_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLG Kernel - Basic function to run on GPU

In [1]:
import numpy as np
from numba import cuda, float64, int64
import math

len_matrix = (100000,3)
len_M0_norm = 100000

@cuda.jit(device=True)
def cross_product(a, b, result):  # Pass result array as an argument
  """
  Calculates the cross product of two 3D vectors.
  """
  result[0] = a[1] * b[2] - a[2] * b[1]
  result[1] = a[2] * b[0] - a[0] * b[2]
  result[2] = a[0] * b[1] - a[1] * b[0]
  #return result  # No need to return, result is modified in-place

@cuda.jit(device=True)
def norm(array):  # Pass result array as an argument
    x = array[0]
    y = array[1]
    z = array[2]
    return math.sqrt(x**2 + y**2 + z**2)

@cuda.jit
#def LLG_kernel(array1, array2, dt, alpha, llg_result, M_0, an_res, bn_res, cn_res, dn_res):
def LLG_kernel(array1, array2, dt, alpha, llg_result):
  llg_gama = gama/((1+alpha**2))
  llg_lamda = gama*alpha/(1+alpha**2)

  cross1 = cuda.local.array(len_matrix, dtype=float64)  # Allocate cross product result arrays
  cross2 = cuda.local.array(len_matrix, dtype=np.float64)
  cross12 = cuda.local.array(len_matrix, dtype=np.float64)
  cross22 = cuda.local.array(len_matrix, dtype=np.float64)
  cross13 = cuda.local.array(len_matrix, dtype=np.float64)
  cross23 = cuda.local.array(len_matrix, dtype=np.float64)
  cross14 = cuda.local.array(len_matrix, dtype=np.float64)
  cross24 = cuda.local.array(len_matrix, dtype=np.float64)
  M_norm = cuda.local.array(len_M0_norm, dtype=np.float64)

  an = cuda.local.array(len_matrix, dtype=np.float64)
  bn = cuda.local.array(len_matrix, dtype=np.float64)
  cn = cuda.local.array(len_matrix, dtype=np.float64)
  dn = cuda.local.array(len_matrix, dtype=np.float64)

  sum_bn = cuda.local.array(len_matrix, dtype=np.float64)
  sum_cn = cuda.local.array(len_matrix, dtype=np.float64)
  sum_dn = cuda.local.array(len_matrix, dtype=np.float64)

  idx = cuda.grid(1)
  if idx < array1.shape[0]:
    # Calculate M0 (norm) manually
    #M_norm[idx] = norm_row(array1[idx])
    temp_norm_result = cuda.local.array(1, dtype=float64)
    temp_norm_result = norm(array1[idx]) # Call norm_row with two arguments
    M_norm[idx] = temp_norm_result

    cross_product(array1[idx], array2[idx], cross1[idx])  # Calculate cross products using modified function
    cross_product(array1[idx], cross1[idx], cross2[idx])

    # Update llg_result directly
    # Modify to use element-wise operations:

    #an = -gma_LL * miu * np.cross(M, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M, np.cross(M, H, axis=1), axis=1)
    #bn = -gma_LL * miu * np.cross(M + (dt / 2) * an, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + (dt / 2) * an, np.cross(M + (dt / 2) * an, H, axis=1), axis=1)
    #cn = -gma_LL * miu * np.cross(M + (dt / 2) * bn, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + (dt / 2) * bn, np.cross(M + (dt / 2) * bn, H, axis=1), axis=1)
    for i in range(3):
        an[idx][i] = -llg_gama * miu * cross1[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross2[idx][i]

    for i in range(3):
        sum_bn[idx][i] = array1[idx][i] + (dt/2) * an[idx][i]

    cross_product(sum_bn[idx], array2[idx], cross12[idx])
    cross_product(sum_bn[idx], cross12[idx], cross22[idx])

    # Modify to use element-wise operations:
    for i in range(3):
        bn[idx][i] = -llg_gama * miu * cross12[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross22[idx][i]

    for i in range(3):
        sum_cn[idx][i] = array1[idx][i] + (dt/2) * bn[idx][i]

    cross_product(sum_cn[idx], array2[idx], cross13[idx])
    cross_product(sum_cn[idx], cross13[idx], cross23[idx])

    for i in range(3):
        cn[idx][i] = -llg_gama * miu * cross13[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross23[idx][i]

    for i in range(3):
        sum_dn[idx][i] = array1[idx][i] + (dt) * cn[idx][i]

    cross_product(sum_dn[idx], array2[idx], cross14[idx])
    cross_product(sum_dn[idx], cross14[idx], cross24[idx])
#   dn = -gma_LL * miu * np.cross(M + dt * cn, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + dt * cn, np.cross(M + dt * cn, H, axis=1), axis=1)

    for i in range(3):
        dn[idx][i] = -llg_gama * miu * cross14[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross24[idx][i]

    for i in range(3):
      llg_result[idx][i] = array1[idx][i] + (dt/6)*(an[idx][i] + 2*bn[idx][i] + 2*cn[idx][i] + dn[idx][i])

    # FOR DEBUG:
    #for i in range(3):
      #an_res[idx][i] = an[idx][i]
      #bn_res[idx][i] = bn[idx][i]
      #cn_res[idx][i] = cn[idx][i]
      #dn_res[idx][i] = dn[idx][i]
    #M_0[idx] = M_norm[idx]


Running the LLG KERNEL for short input arrays (len 3)

> Add blockquote



In [20]:
import numpy as np
from numba import cuda, float64, int64
import math

#physical parameters
eps=8.854e-12 #[F/m]
miu=4*np.pi*1e-7 #[H/m]
c=1/(eps*miu)**0.5
heta = (miu/eps)**0.5
q = 1.60217646e-19    # Elementary charge [Coulombs]
miu = 4 * np.pi * 1e-7    # Magnetic permeability [H/m]
g = 2    # Landau factor
me = 9.1093821545e-31    # Electron mass [kg]
gma_factor = 1
gama = gma_factor * g * q / (2 * me)
alpha = 0

dz = 2e-9/8
dt = 2

x = np.array([[1.,2.,3.], [4.,5.,6.], [7.,2.,5.]], dtype = np.float64)
#norm_x = np.array(np.linalg.norm(x, axis=1))
y = np.array([[4,5,6], [1,7,3], [4,5,6]], dtype = np.float64)
M_norm = np.arange(3, dtype=np.float64)
#an_res = np.arange(3, dtype=np.float64)
#d_x = cuda.to_device(x)
#d_y = cuda.to_device(y)
res = np.empty_like(x)
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_res = cuda.device_array_like(d_x)

# FOR DEBUG - uncomment and print to understand better:
#d_an_res = cuda.device_array_like(d_x)
#d_bn_res = cuda.device_array_like(d_x)
#d_cn_res = cuda.device_array_like(d_x)
#d_dn_res = cuda.device_array_like(d_x)
#d_M_norm = cuda.device_array_like(M_norm)

blocks = 8  # Ensure enough blocks to cover the data
threadsperblock = 64 # Ensure enough threads to cover the data

# Call the kernel with launch configuration
# FOR DEBUG VERSION (with M0, an, bn...):
#LLG_kernel[blocks, threadsperblock](d_x, d_y, dt, alpha, d_res, d_M_norm, d_an_res, d_bn_res, d_cn_res, d_dn_res)

LLG_kernel[blocks, threadsperblock](d_x, d_y, dt, alpha, d_res)
print("res:",d_res.copy_to_host())
#print("M0:",d_M_norm.copy_to_host())
#print("an:",d_an_res.copy_to_host())
#print("bn:",d_bn_res.copy_to_host())
#print("cn:",d_cn_res.copy_to_host())
#print("dn:",d_dn_res.copy_to_host())

ImportError: Minor version compatibility requires ptxcompiler and cubinlinker packages to be available

In [16]:
!nvidia-smi
!nvcc --version

Sun Feb 16 09:21:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   61C    P0             30W /   72W |     215MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [17]:
!pip install --upgrade numba-cuda

In [ ]:
%timeit LLG_kernel[4,16](d_x, d_y, dt, alpha, d_res); cuda.synchronize()

/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/dispatcher.py:605: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


67.7 µs ± 661 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [3]:
!uv pip install -q --system --force-reinstall numba-cuda==0.4.0

In [4]:
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [19]:
from numba import config
config.CUDA_ENABLE_MINOR_VERSION_COMPATIBILITY = True

LLG Step - The basic function to run on CPU

In [6]:
import numpy as np

def LLG_step(M: np.array, H: np.array, dt: float, alpha: float) -> np.array:
    """
    """
    M0 = np.linalg.norm(M, axis=1, keepdims=True)
    #print("M0:", M0)
    gma_LL=gama/((1+alpha**2))
    LL_lambda=gama*alpha/(1+alpha**2)

   # Compute LLG terms
    an = -gma_LL * miu * np.cross(M, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M, np.cross(M, H, axis=1), axis=1)
    #print("an:",an)
    bn = -gma_LL * miu * np.cross(M + (dt / 2) * an, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + (dt / 2) * an, np.cross(M + (dt / 2) * an, H, axis=1), axis=1)
    #print("bn:",bn)
    cn = -gma_LL * miu * np.cross(M + (dt / 2) * bn, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + (dt / 2) * bn, np.cross(M + (dt / 2) * bn, H, axis=1), axis=1)
    #print("cn:",cn)
    dn = -gma_LL * miu * np.cross(M + dt * cn, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + dt * cn, np.cross(M + dt * cn, H, axis=1), axis=1)
    #print("dn:",dn)
    new_M = M + (dt/6)*(an+2*bn+2*cn+dn)
    return new_M

Running the LLG on CPU for short input arrays (len 3)

In [7]:
import numpy as np
from numba import cuda
import math

#physical parameters
eps=8.854e-12 #[F/m]
miu=4*np.pi*1e-7 #[H/m]
c=1/(eps*miu)**0.5
heta = (miu/eps)**0.5
q = 1.60217646e-19    # Elementary charge [Coulombs]
miu = 4 * np.pi * 1e-7    # Magnetic permeability [H/m]
g = 2    # Landau factor
me = 9.1093821545e-31    # Electron mass [kg]
gma_factor = 1
gama = gma_factor * g * q / (2 * me)
alpha = 0

dz = 2e-9/8
dt = 2

x = np.array([[1.,2.,3.], [4.,5.,6.], [7.,2.,5.]], dtype = np.float64)
y = np.array([[4,5,6], [1,7,3], [4,5,6]], dtype = np.float64)
#d_x = cuda.to_device(x)
#d_y = cuda.to_device(y)
res = np.empty_like(x)

LLG_step(x, y, dt, alpha)


array([[-6.24733836e+24, -7.34973942e+23,  4.77737052e+24],
       [ 1.68011243e+25, -9.76156552e+24,  1.71766114e+25],
       [ 3.27066219e+25, -2.27844227e+25, -2.81739568e+24]])

In [8]:
%timeit LLG_step(x, y, dt, alpha)

363 µs ± 39.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Adding the time propogate loop - Where in each iteration there is a call for the LLG Kernel. Suppose to be less time effictiate beacuse in each iterate there is a data transfer for and to host and device.

In [9]:
import numpy as np
from numba import cuda, float64, int64
import math

#physical parameters
eps=8.854e-12 #[F/m]
miu=4*np.pi*1e-7 #[H/m]
c=1/(eps*miu)**0.5
heta = (miu/eps)**0.5
q = 1.60217646e-19    # Elementary charge [Coulombs]
miu = 4 * np.pi * 1e-7    # Magnetic permeability [H/m]
g = 2    # Landau factor
me = 9.1093821545e-31    # Electron mass [kg]
gma_factor = 1
gama = gma_factor * g * q / (2 * me)
alpha = 0
dz = 2e-9/8
dt = 2

big_arr1 = np.arange(0,2*100000*3,2).reshape((100000, 3))
big_arr2 = np.arange(100000*3).reshape((100000, 3))

time_steps = 100
d_big_arr1 = cuda.to_device(big_arr1)
d_big_arr2 = cuda.to_device(big_arr2)
M_norm = np.empty_like(big_arr1)
res = np.empty_like(big_arr1)
d_res = cuda.device_array_like(big_arr1)
saved_res_for_alltime = np.zeros(shape=(time_steps*100000,3))

blocks = 8  # Ensure enough blocks to cover the data
threadsperblock = 256 # Ensure enough threads to cover the data


print("arr1",big_arr1)
print("arr2",big_arr2)
print("res:",saved_res_for_alltime)

arr1 [[     0      2      4]
 [     6      8     10]
 [    12     14     16]
 ...
 [599982 599984 599986]
 [599988 599990 599992]
 [599994 599996 599998]]
arr2 [[     0      1      2]
 [     3      4      5]
 [     6      7      8]
 ...
 [299991 299992 299993]
 [299994 299995 299996]
 [299997 299998 299999]]
res: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [10]:
import numpy as np

def LLG_in_loop(big_arr1: np.array, big_arr2: np.array, dt: int, alpha: int, blocks: int,
                threadsperblock: int ,saved_res_for_alltime: np.array) -> np.array:
  d_big_arr1 = cuda.to_device(big_arr1)
  d_big_arr2 = cuda.to_device(big_arr2)
  for i in range(0,time_steps):
    LLG_kernel[blocks, threadsperblock](d_big_arr1, d_big_arr2, dt, alpha, d_res)
    print("d_res",d_res.copy_to_host())
    saved_res_for_alltime[i*100000, i*100000,i*100000] = d_res.copy_to_host()  # Assign to index i

In [11]:
%timeit LLG_in_loop(d_big_arr1, d_big_arr2, dt, alpha, blocks, threadsperblock,saved_res_for_alltime); cuda.synchronize()

/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  if ntasks < 0:
ERROR:numba.cuda.cudadrv.driver:Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION


LinkerError: [222] Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION
ptxas application ptx input, line 9; fatal   : Unsupported .version 8.5; current version is '8.4'

In [12]:
import numpy as np
from numba import cuda, float64, int64
import math

#physical parameters
eps=8.854e-12 #[F/m]
miu=4*np.pi*1e-7 #[H/m]
c=1/(eps*miu)**0.5
heta = (miu/eps)**0.5
q = 1.60217646e-19    # Elementary charge [Coulombs]
miu = 4 * np.pi * 1e-7    # Magnetic permeability [H/m]
g = 2    # Landau factor
me = 9.1093821545e-31    # Electron mass [kg]
gma_factor = 1
gama = gma_factor * g * q / (2 * me)
alpha = 0
dz = 2e-9/8
dt = 2

big_arr1 = np.arange(0,2*100000*3,2).reshape((100000, 3))
big_arr2 = np.arange(100000*3).reshape((100000, 3))

time_steps = 100
d_big_arr1 = cuda.to_device(big_arr1)
d_big_arr2 = cuda.to_device(big_arr2)
M_norm = np.empty_like(big_arr1)
res = np.empty_like(big_arr1)
d_res = cuda.device_array_like(big_arr1)
saved_res_for_alltime = np.zeros(shape=(time_steps,100000,3))

blocks = 8  # Ensure enough blocks to cover the data
threadsperblock = 256 # Ensure enough threads to cover the data

def loop_in_LLG(big_arr1, big_arr2, dt, alpha, blocks, threadsperblock):
  d_big_arr1 = cuda.to_device(big_arr1)
  d_big_arr2 = cuda.to_device(big_arr2)
  for i in range(0,time_steps):
    LLG_kernel[blocks, threadsperblock](d_big_arr1, d_big_arr2, dt, alpha, d_res)
    saved_res_for_alltime[i, :,:] = d_res.copy_to_host()  # Assign to index i

print("arr1",big_arr1)
print("arr2",big_arr2)
print("res:",d_res.copy_to_host())

arr1 [[     0      2      4]
 [     6      8     10]
 [    12     14     16]
 ...
 [599982 599984 599986]
 [599988 599990 599992]
 [599994 599996 599998]]
arr2 [[     0      1      2]
 [     3      4      5]
 [     6      7      8]
 ...
 [299991 299992 299993]
 [299994 299995 299996]
 [299997 299998 299999]]
res: [[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [13]:
import numpy as np
from numba import cuda, float64, int64
import math

len_matrix = (100000,3)
len_M0_norm = 100000

@cuda.jit(device=True)
def cross_product(a, b, result):  # Pass result array as an argument
  """
  Calculates the cross product of two 3D vectors.
  """
  result[0] = a[1] * b[2] - a[2] * b[1]
  result[1] = a[2] * b[0] - a[0] * b[2]
  result[2] = a[0] * b[1] - a[1] * b[0]
  #return result  # No need to return, result is modified in-place

@cuda.jit(device=True)
def norm(array):  # Pass result array as an argument
    x = array[0]
    y = array[1]
    z = array[2]
    return math.sqrt(x**2 + y**2 + z**2)

@cuda.jit
#def LLG_kernel(array1, array2, dt, alpha, llg_result, M_0, an_res, bn_res, cn_res, dn_res):
def Loop_in_LLG_kernel(array1, array2, dt, alpha, llg_result, n_times):
  for t in range(0, n_times):
    llg_gama = gama/((1+alpha**2))
    llg_lamda = gama*alpha/(1+alpha**2)

    cross1 = cuda.local.array(len_matrix, dtype=float64)  # Allocate cross product result arrays
    cross2 = cuda.local.array(len_matrix, dtype=np.float64)
    cross12 = cuda.local.array(len_matrix, dtype=np.float64)
    cross22 = cuda.local.array(len_matrix, dtype=np.float64)
    cross13 = cuda.local.array(len_matrix, dtype=np.float64)
    cross23 = cuda.local.array(len_matrix, dtype=np.float64)
    cross14 = cuda.local.array(len_matrix, dtype=np.float64)
    cross24 = cuda.local.array(len_matrix, dtype=np.float64)
    M_norm = cuda.local.array(len_M0_norm, dtype=np.float64)

    an = cuda.local.array(len_matrix, dtype=np.float64)
    bn = cuda.local.array(len_matrix, dtype=np.float64)
    cn = cuda.local.array(len_matrix, dtype=np.float64)
    dn = cuda.local.array(len_matrix, dtype=np.float64)

    sum_bn = cuda.local.array(len_matrix, dtype=np.float64)
    sum_cn = cuda.local.array(len_matrix, dtype=np.float64)
    sum_dn = cuda.local.array(len_matrix, dtype=np.float64)

    idx = cuda.grid(1)
    if idx < array1.shape[0]:
      # Calculate M0 (norm) manually
      #M_norm[idx] = norm_row(array1[idx])
      temp_norm_result = cuda.local.array(1, dtype=float64)
      temp_norm_result = norm(array1[idx]) # Call norm_row with two arguments
      M_norm[idx] = temp_norm_result

      cross_product(array1[idx], array2[idx], cross1[idx])  # Calculate cross products using modified function
      cross_product(array1[idx], cross1[idx], cross2[idx])

      # Update llg_result directly
      # Modify to use element-wise operations:

      #an = -gma_LL * miu * np.cross(M, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M, np.cross(M, H, axis=1), axis=1)
      #bn = -gma_LL * miu * np.cross(M + (dt / 2) * an, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + (dt / 2) * an, np.cross(M + (dt / 2) * an, H, axis=1), axis=1)
      #cn = -gma_LL * miu * np.cross(M + (dt / 2) * bn, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + (dt / 2) * bn, np.cross(M + (dt / 2) * bn, H, axis=1), axis=1)
      for i in range(3):
          an[idx][i] = -llg_gama * miu * cross1[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross2[idx][i]

      for i in range(3):
          sum_bn[idx][i] = array1[idx][i] + (dt/2) * an[idx][i]

      cross_product(sum_bn[idx], array2[idx], cross12[idx])
      cross_product(sum_bn[idx], cross12[idx], cross22[idx])

      # Modify to use element-wise operations:
      for i in range(3):
          bn[idx][i] = -llg_gama * miu * cross12[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross22[idx][i]

      for i in range(3):
          sum_cn[idx][i] = array1[idx][i] + (dt/2) * bn[idx][i]

      cross_product(sum_cn[idx], array2[idx], cross13[idx])
      cross_product(sum_cn[idx], cross13[idx], cross23[idx])

      for i in range(3):
          cn[idx][i] = -llg_gama * miu * cross13[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross23[idx][i]

      for i in range(3):
          sum_dn[idx][i] = array1[idx][i] + (dt) * cn[idx][i]

      cross_product(sum_dn[idx], array2[idx], cross14[idx])
      cross_product(sum_dn[idx], cross14[idx], cross24[idx])
  #   dn = -gma_LL * miu * np.cross(M + dt * cn, H, axis=1) - (LL_lambda * miu / M0) * np.cross(M + dt * cn, np.cross(M + dt * cn, H, axis=1), axis=1)

      for i in range(3):
          dn[idx][i] = -llg_gama * miu * cross14[idx][i] - (llg_lamda * miu / M_norm[idx]) * cross24[idx][i]

      for i in range(3):
        llg_result[idx][i] = array1[idx][i] + (dt/6)*(an[idx][i] + 2*bn[idx][i] + 2*cn[idx][i] + dn[idx][i])

    # FOR DEBUG:
    #for i in range(3):
      #an_res[idx][i] = an[idx][i]
      #bn_res[idx][i] = bn[idx][i]
      #cn_res[idx][i] = cn[idx][i]
      #dn_res[idx][i] = dn[idx][i]
    #M_0[idx] = M_norm[idx]
  saved_res_for_alltime[t, :,:] = llg_result[]  # Assign to index i

SyntaxError: invalid syntax (<ipython-input-13-136d260c1ffc>, line 110)

In [15]:
def pulse_gen(max_feald: float, fea : float, FWHM: float, dt : float ,lamda = 800e-9 , teta = 0, start_sefty = 4 ):
    """
    input-

    """
    # move to time w
    w = 2* np.pi * c/lamda
    teta -= np.pi/4
    turn_m= np.array([np.cos(teta) , -np.sin(teta), np.sin(teta) ,np.cos(teta) ])
    turn_m = turn_m.reshape((2,2))
    pulse = max_feald * turn_m @ np.array([1 ,np.exp(-fea*1j)]).reshape(2)
    sigma = ((0.5/np.log(2))** 0.5) * FWHM
    peak_location = start_sefty*sigma

    def out_pulse(n: int) -> np.array:
        """
        input-
        :patam n: index of step in time
        :return: 2D comlmx np.array of the inject pulse (E_x E_y) in this time step
        """
        r = pulse  * np.exp(- (n*dt - peak_location)**2/( 2*sigma**2))* np.exp(1j*w*n*dt)
        return r
    return out_pulse


In [ ]:
# magnets matirals proprety

# cobalt
Co_er = 1
Co_sigma = 1.7e7 #https://periodictable.com/Elements/027/data.html
Co_er_complex= -16 + 23.3j

# Iron
Fe_er = 1
Fe_sigma = 1e7

# nical
Ni_er = 1
Ni_sigma = 1.4e7 #https://periodictable.com/Elements/027/data.html

alpae_renge = (0.025 ,0.0025) # genral range
gilbert_damping_time = 600e-12 # [sec] artical - concting to alpha in {I dont remmber how}
precession_frequency = 8.5e9 #[Hz] artical

# aditinal matirals proprety
Pt_er = 1
Pt_sigma = 7e6 # [1/oham*m]

Au_er = 1
Au_sigma = 1e6 # (spintronics lab)


alpha = {"Co_Au" : 0.02, "Co_Pt" : 0.025 , "Fe_Au": 0.02, "Fe_Pt" : 0.025, "Ni_Au": 0.05, "Ni_Pt" : 0.06}

key_name = [
            "H times picture",
            "E times picture",
            "ms time picture",
            "time picture",
            "H in locations",
            "E in locations",
            "ms in locations",
            "locations",
            "time intervals",
            "matitral location", "e_r" ,"conductivity", "gilbert damping factor","initial magnetization",
            "max magnetic field" ,"polarization phase","pulse width (FWHM)",
            "systen status", "dt", "safety_start"
            ]

In [ ]:
# pulse proprty (in the article)
mean_wave_lenght= 784e-9 # [m]
# B_opt =  [0.02, 0.2 ] # [T]
# FWHM = 1.1e-12 #[sec]
FWHM = 1.1e-12 #[sec]
B_opt =2.4e5 #[A/m ] = 0.3[T]

w= 2*np.pi/(mean_wave_lenght/sim.c)

path = "C:\maxwell-LLG\ws\simulation_result\Full_Width_Pulse\\"




# besic stuff
time_cycal= mean_wave_lenght/sim.c
dz = 2e-9/8
dt = dz/(2*sim.c)

# matiral length according to arcticle
magntic_steps = 10e-9//dz
conductive_steps = 2e-9//dz

# pulse properties
sefe_start = 3
sigma_pulse = 0.5* (np.log(2)**-0.5) * FWHM
peak_enter_time= sefe_start * sigma_pulse
fie = np.pi/2

#material properties
magntic_name = ["Co", "Fe", "Ni"]
conductive_name = ["Au", "Pt"]

magntic_eps = [Co_er, Fe_er, Ni_er ]
conductive_eps = [  Au_er, Pt_er]

magntic_sigma = [Co_sigma, Fe_sigma, Ni_sigma ]
conductive_sigma = [ Au_sigma, Pt_sigma]

ms0 = 3e5*np.array([1, 0, 0 , 0 ,0, 0.001]).reshape((2,3))

#simulation building blocks
z1 = 8
# z1 = 20
z_end = int(z1+ magntic_steps + conductive_steps)
z_indexes = [z1, int(z1+ magntic_steps), z_end ]


save_loc = np.arange(z1-1, z_end+1)
save_loc[0] = 4
save_time= []


void_steps = 2*z1

exit_steps = int(time_cycal/ dt)


# we run in sorter time but this what we need:
simulation_time = int(2.9*(peak_enter_time//dt + void_steps +4* (magntic_steps + conductive_steps)) + exit_steps)


In [ ]:
import numpy as np
from tqdm import tqdm

z_ind = [7,100,555]
e_r = [1,1,1]
sigma = [1.7e7,1e7,1.4e7] # Co, Iron, Nical
alpha = {"Co_Au" : 0.02, "Co_Pt" : 0.025 , "Fe_Au": 0.02, "Fe_Pt" : 0.025, "Ni_Au": 0.05, "Ni_Pt" : 0.06}
m0 =



def simulation(z_ind: list[int], e_r : list[float], sigma: list[float], alpha: float, m0: np.array,
               max_E: float, fea : float, FWHM: float,
               save_location, save_time, close_system: bool,
               dt: float, simulation_time_steps: int, lamda = 800e-9 , teta = 0, safety_start = 4, time_interval = 1):
    """
    inputs-
    :param z1: start index of material
    :param z2: end index of material
    :param e_r:  relative permittivity
    :param sigma: condactivity of matiral [Oham/m]
    :param alpha: deeiha param of moments
    :param m0: initial BIG MAGNETIZATION (sum of moments) configuration in [x,y,z] -> vector size 3
    :param max_E: max value of the fileds [A/m]
    :param fea: phase defernt betwin E_x and E_y
    :param FWHM: the whith of the gausian , time when the pulse is greater than half max power [sec].
    :patm save_location: interbel of index in space to save all value in the for all time.
    :patm save_time: interbel of index in time to save all value at the intaer space.
    :param close_system: wether will use the reflection of M on H on E or not.
    :param dt: time step size [s]
    :param simulation_time_steps: numer for time steps to do [int]
    :param lamda: amplifid wave lenght [m]
    :param teta: angle of the central axis of the polarization [rad]

    :return: two list in len 3
        first list - save parmaters in all space in the save_time stepes
        secnd list - saved paarmates in all time in the seve_location ind on space
        parm are: [H, E, msi ] ( msi given only in the relvent space if axis)
                    all of them in the shep of [time_step]* [space_step]* [3 demntion]
    """


    # system defntion
    dz = 2*c *dt
    lenz= z_ind[-1] + int(lamda/dz)


    # matrial proprty
    const_for_les_E = np.ones(lenz-1)
    const_for_H = 0.5*np.ones(lenz-1)
    for i in range(len(e_r)):
        eps_r = np.ones(z_ind[i+1] - z_ind[i])
        eps_r += (e_r[i]-1)
        eaf = dt/2/eps* (sigma[i]) /eps_r
        const_for_les_E[z_ind[i]:z_ind[i+1]] = ((1-eaf)/(1+eaf))
        const_for_H[z_ind[i]:z_ind[i+1]] = (0.5/eps_r/(1+eaf))

    # plase for run the symolation
    pulse = pulse_gen(max_E, fea, FWHM, dt, lamda, teta, safety_start)

    # defing the worke spaec for the simltion
    E_next= np.zeros((lenz, 3)).astype(complex)
    E_current= np.zeros((lenz, 3)).astype(complex)
    right_boundry = [np.zeros((3)), np.zeros((3))]
    left_boundry= [np.zeros((3)), np.zeros((3))]
    H_next= np.zeros((lenz, 3)).astype(complex)
    H_current= np.zeros((lenz, 3)).astype(complex)

    ms_i_current= np.zeros((z_ind[-1] -z_ind[0], 3)) ## in every place there is matirial
    ms_i_next= np.zeros((z_ind[-1] -z_ind[0], 3)) ## will be filled in the propogate loop acoording to LLG

    # intial the beging magntiztion defending on M0 and the index of matiral
    for i in range(len(e_r)):
        ms_us = m0[i,:] ## : means i want to see all 3 axiz
        st_ind = z_ind[i]- z_ind[0]
        end_ind = z_ind[i+1] - z_ind[0]
        for k in range(3):
            ms_i_current[st_ind:end_ind,k]= ms_us[k]/(end_ind-st_ind)


    # defing rutenting array:
    # for space indxes
    time_save = int(simulation_time_steps/time_interval + 1)
    E_space_return = np.zeros((time_save, len(save_location), 2)).astype(complex)
    H_space_return = np.zeros((time_save, len(save_location), 2)).astype(complex)
    msi_space_return = np.zeros((time_save, z_ind[-1]-z_ind[0],3)).astype(complex)

    # for time indxes
    E_time_return = np.zeros((len(save_time), lenz, 2)).astype(complex)
    H_time_return = np.zeros((len(save_time), lenz, 2)).astype(complex)
    msi_time_return = np.zeros((len(save_time), z_ind[-1]-z_ind[0] , 3)).astype(complex)
    index_to_save_in =0

    # propegate

    for n in tqdm(range(0, simulation_time_steps), desc= "simulation pross: "):
        E_current[ 3,0:2] += pulse(n) # inject the input signal

        # set bondry condition for not replction
        r = right_boundry.pop(0)
        l = left_boundry.pop(0)

        E_current[ -1,:] = r # last step the end is r condotion
        E_current[0,:] = l # same to left

        # H eqution
        H_next[1:, 0]= H_current[1:, 0]+ 0.5*(E_current[1:,1] - E_current[:-1, 1] )
        H_next[1:, 1]= H_current[1:, 1]- 0.5*(E_current[1:, 0] - E_current[:-1, 0] )
        # LLG and cang in the ms_i
        LLG_kernel[blocks, threadsperblock](ms_i_current, np.real(H_current[z_ind[0]:z_ind[-1],:]), dt, alpha, ms_i_next)
        if close_system: # intagrtion line for close sistem B = H + M
            H_next[ z_ind[0]:z_ind[-1], :] += ms_i_current - ms_i_next
        # E eqution
        E_next[ :-1, 1] = const_for_les_E*E_current[:-1,1] + const_for_H* (H_next[1:,0] - H_next[ :-1, 0])
        E_next[ :-1, 0] = const_for_les_E*E_current[:-1, 0] - const_for_H* (H_next[1:, 1] - H_next[ :-1, 1])


        # get redy to next time step
        right_boundry.append(np.copy(E_current[ -2,:]))
        left_boundry.append(np.copy(E_current[ 1,:]))
        E_current = np.copy(E_next)
        H_current = np.copy(H_next)
        ms_i_current = np.copy(ms_i_next)

        # save to return
        # save picture of parameters in time
        if n in save_time:
            H_time_return[index_to_save_in, :,:] = H_current[:,:2]
            E_time_return[index_to_save_in, :,:] = E_current[:,:2]
            msi_time_return[index_to_save_in, :,:] = ms_i_current

            index_to_save_in += 1

        # save the parameters in speceific location in all times
        if not n % time_interval :
            p =int(n/time_interval)
            H_space_return[p, :, :] = H_current[save_location, :2]
            E_space_return[p, :, :] = E_current[save_location, :2]
            msi_space_return[p, :, :] = ms_i_current
    # time_r = H_time_return, E_time_return, msi_time_return
    # spece_r = H_space_return, E_space_return, msi_space_return
    all_data = {
                "H times picture": H_time_return,
                "E times picture": E_time_return,
                "ms time picture": msi_time_return,
                "time picture": save_time,
                "H in locations": H_space_return,
                "E in locations": E_space_return,
                "ms in locations": msi_space_return,
                "locations": save_location,
                "time intervals": time_interval,
                "matitral location": z_ind, "e_r" : e_r ,"conductivity": sigma, "gilbert damping factor": alpha,"initial magnetization": m0,
                "max magnetic field" :max_E ,"polarization phase": fea,"pulse width (FWHM)": FWHM, "mean wave lenght" : lamda,
                "systen status": close_system, "dt": dt, "safety_start": safety_start
                }
    return  all_data

In [ ]:
%timeit LLG_kernel[blocks,threadsperblock](d_big_arr1, d_big_arr2, dt, alpha, d_res); cuda.synchronize()

104 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
import numpy as np
from numba import cuda, float64, int64
import math

#physical parameters
eps=8.854e-12 #[F/m]
miu=4*np.pi*1e-7 #[H/m]
c=1/(eps*miu)**0.5
heta = (miu/eps)**0.5
q = 1.60217646e-19    # Elementary charge [Coulombs]
miu = 4 * np.pi * 1e-7    # Magnetic permeability [H/m]
g = 2    # Landau factor
me = 9.1093821545e-31    # Electron mass [kg]
gma_factor = 1
gama = gma_factor * g * q / (2 * me)
alpha = 0
dz = 2e-9/8
dt = 2

big_arr1 = np.arange(0,2*100000*3,2).reshape((100000, 3))
big_arr2 = np.arange(100000*3).reshape((100000, 3))

#d_big_arr1 = cuda.to_device(big_arr1)
#d_big_arr2 = cuda.to_device(big_arr2)
M_norm = np.empty_like(big_arr1)
res = np.empty_like(big_arr1)
#d_res = cuda.device_array_like(big_arr1)

blocks = 16  # Ensure enough blocks to cover the data
threadsperblock = 512 # Ensure enough threads to cover the data

print("arr1",big_arr1)
print("arr2",big_arr2)
LLG_step(big_arr1, big_arr2, dt, alpha)


arr1 [[     0      2      4]
 [     6      8     10]
 [    12     14     16]
 ...
 [599982 599984 599986]
 [599988 599990 599992]
 [599994 599996 599998]]
arr2 [[     0      1      2]
 [     3      4      5]
 [     6      7      8]
 ...
 [299991 299992 299993]
 [299994 299995 299996]
 [299997 299998 299999]]


array([[0.00000e+00, 2.00000e+00, 4.00000e+00],
       [6.00000e+00, 8.00000e+00, 1.00000e+01],
       [1.20000e+01, 1.40000e+01, 1.60000e+01],
       ...,
       [5.99982e+05, 5.99984e+05, 5.99986e+05],
       [5.99988e+05, 5.99990e+05, 5.99992e+05],
       [5.99994e+05, 5.99996e+05, 5.99998e+05]])

In [ ]:
%timeit LLG_step(big_arr1, big_arr2, dt, alpha)

68.5 ms ± 6.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
